

Creamos un dataframe con marcas de coches y sus países de origen



In [1]:
import pandas as pd
import numpy as np
nombremarcas1 = ['Renault','Toyota','Ford']
paises1 = ["Francia","Japón", "USA"]


marcas1 = pd.DataFrame(nombremarcas1, columns = ['Marca'])
marcas1["Pais"] = paises1
print (marcas1)

     Marca     Pais
0  Renault  Francia
1   Toyota    Japón
2     Ford      USA


Otro igual con otros datos

In [2]:
nombremarcas2 = ['Citroen','Nissan','GM']
paises2 = ["Francia","Japón", "USA"]

marcas2 = pd.DataFrame(nombremarcas2, columns = ['Marca'])
marcas2["Pais"] = paises2
print(marcas2)

     Marca     Pais
0  Citroen  Francia
1   Nissan    Japón
2       GM      USA


Unirlos en uno que tenga las 6 filas

In [5]:
marcas = pd.concat([marcas1, marcas2], axis = 0, ignore_index= True)
marcas

,Marca,Pais
0,Renault,Francia
1,Toyota,Japón
2,Ford,USA
3,Citroen,Francia
4,Nissan,Japón
5,GM,USA


Join
Empezamos creando un nuevo dataframe con algunos datos en común con el anterior


In [ ]:
empleados_dict = {'Marca': ['Renault', 'Toyota', 'Ford','Citroen','Subaru', 'Skoda'],
                  'Empleados' :[10000, 15000, 20000, 25000, 30000, 40000]}
empleados = pd.DataFrame(empleados_dict)
empleados

,Marca,Empleados
0,Renault,10000
1,Toyota,15000
2,Ford,20000
3,Citroen,25000
4,Subaru,30000
5,Skoda,40000


Para unirlos en un solo dataframe con las tres columnas, se puede usar merge()

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.merge.html#pandas.merge
pd.merge(marcas, empleados)
#equivalentes
#pd.merge(marcas, empleados, on = "Marca")
#pd.merge(marcas, empleados, left_on = "Marca", right_on = "Marca")

,Marca,Pais,Empleados
0,Renault,Francia,10000
1,Toyota,Japón,15000
2,Ford,USA,20000
3,Citroen,Francia,25000


Para probar el merge por índice, añadimos un índice a los dataframes, va a ser un supuesto código de marca


In [ ]:
marcascodigo = marcas.copy()
# inventamos unos codigos para la tabal de marcas
codigos = ['c1','c2','c3','c4','c5','c6']
marcascodigo.index = codigos
marcascodigo

,Marca,Pais
c1,Renault,Francia
c2,Toyota,Japón
c3,Ford,USA
c4,Citroen,Francia
c5,Nissan,Japón
c6,GM,USA


Lo mismo con el dataframe de empleados, vigilando que los códigos cuadren

In [ ]:
empleadoscodigo = empleados.copy()
codigos = ['c1','c2','c3','c4','c7','c8']
empleadoscodigo.index = codigos
empleadoscodigo

,Marca,Empleados
c1,Renault,10000
c2,Toyota,15000
c3,Ford,20000
c4,Citroen,25000
c7,Subaru,30000
c8,Skoda,40000


In [ ]:
pd.merge(marcascodigo, empleadoscodigo, left_index = True, right_index=True)

,Marca_x,Pais,Marca_y,Empleados
c1,Renault,Francia,Renault,10000
c2,Toyota,Japón,Toyota,15000
c3,Ford,USA,Ford,20000
c4,Citroen,Francia,Citroen,25000


# Agrupar

In [ ]:
tabla = pd.merge(marcascodigo, empleadoscodigo, left_index = True, right_index=True)
tabla.drop(columns = "Marca_y", inplace = True)
tabla.rename(columns={"Marca_x": "Marca"}, inplace = True)
tabla


,Marca,Pais,Empleados
c1,Renault,Francia,10000
c2,Toyota,Japón,15000
c3,Ford,USA,20000
c4,Citroen,Francia,25000


Por defecto, groupby devuelve los datos en una serie indexada con los grupos.

In [ ]:
numMarcas = tabla.groupby("Pais")["Marca"].count()
numMarcas

Pais
Francia    2
Japón      1
USA        1
Name: Marca, dtype: int64

In [ ]:
numMarcas['USA']

np.int64(1)

Para recorrer los grupos

In [ ]:
grupos = tabla.groupby("Pais")

In [ ]:
for pais, datos in grupos:
  print(pais)
  print(datos.head())

Francia
      Marca     Pais  Empleados
c1  Renault  Francia      10000
c4  Citroen  Francia      25000
Japón
     Marca   Pais  Empleados
c2  Toyota  Japón      15000
USA
   Marca Pais  Empleados
c3  Ford  USA      20000


Para acceder a un grupo en concreto

In [ ]:
grupos.get_group("Francia")

,Marca,Pais,Empleados
c1,Renault,Francia,10000
c4,Citroen,Francia,25000


In [ ]:
numEmple = tabla.groupby("Pais")["Empleados"].sum()
numEmple

Pais
Francia    35000
Japón      15000
USA        20000
Name: Empleados, dtype: int64

Con as_index=False devuelve un dataframe

In [ ]:
tabla2 = tabla.copy()
tabla2['Ventas'] = np.linspace(100000, 200000, 4)
# equivalente
# tabla2['Ventas'] = [100000 + x * 100000/3 for x in range(4)]
tabla2

,Marca,Pais,Empleados,Ventas
c1,Renault,Francia,10000,100000.000000
c2,Toyota,Japón,15000,133333.333333
c3,Ford,USA,20000,166666.666667
c4,Citroen,Francia,25000,200000.000000


In [ ]:
numMarcas2 = tabla.groupby("Pais",as_index=False)["Marca"].count()
numMarcas2

,Pais,Marca
0,Francia,2
1,Japón,1
2,USA,1


Para calcular más de una función agregada

In [ ]:
numMarcas2 = tabla2.groupby("Pais")["Empleados"].aggregate(["sum", "mean", "count"])
numMarcas2

,sum,mean,count
Pais,,,
Francia,35000,17500.0,2
Japón,15000,15000.0,1
USA,20000,20000.0,1


In [ ]:
numMarcas2.loc["Francia"]


sum      35000.0
mean     17500.0
count        2.0
Name: Francia, dtype: float64

Si no se selecciona columna, se aplican las funciones agregadas a todas las columnas.

In [ ]:
numMarcas2 = tabla2.groupby("Pais").aggregate(["sum", "count"])
numMarcas2
# atención: concatena los nombre de los países, por la suma

Marca       Empleados               Ventas      
                    sum count       sum count            sum count
Pais                                                              
Francia  RenaultCitroen     2     35000     2  300000.000000     2
Japón            Toyota     1     15000     1  133333.333333     1
USA                Ford     1     20000     1  166666.666667     1

Podemos acceder s cada grupo. Salen todas las columnas con sus funciones agregadas.

In [ ]:
numMarcas2.loc["Francia"]

Marca      sum      RenaultCitroen
           count                 2
Empleados  sum               35000
           count                 2
Ventas     sum            300000.0
           count                 2
Name: Francia, dtype: object

Podemos acceder a una columna de un grupo. Salen las funciones agregadas.

In [ ]:
numMarcas2.loc["Francia"]["Ventas"]

sum      300000.0
count           2
Name: Francia, dtype: object

Para acceder a un valor concreto hay que poner tres índices.

In [ ]:
numMarcas2.loc["Francia"]["Ventas"]["sum"]

np.float64(300000.0)

# value_counts()

Para ciertos casos, puede ser más práctico value_counts()

In [ ]:
marcas.Pais.value_counts()

Pais
Francia    2
Japón      2
USA        2
Name: count, dtype: int64